In [1]:
# imports
import pennylane as qml
import numpy as np
from pennylane_snowflurry.API.api_client import MonarqClient
import pennylane_snowflurry.utility.test_circuits as circuits
import os
from pennylane_snowflurry.transpiler.monarq_transpile import Transpiler
from pennylane_snowflurry.transpiler.transpiler_config import MonarqDefaultConfig, TranspilerConfig
from pennylane_snowflurry.utility.debug_utility import arbitrary_circuit
from pennylane_snowflurry.monarq_device import MonarqDevice

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


c:\Users\bouch\AppData\Local\Programs\Python\Python312\Lib\site-packages\pennylane\__init__.py:200: PennyLaneDeprecationWarning: Device will no longer be accessible top level. Please access the class as pennylane.devices.LegacyDevice
  warn(


In [2]:
# 6-ghz circuit
def ghz6():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.CNOT([0, 2])
    qml.CNOT([0, 3])
    qml.CNOT([0, 4])
    qml.CNOT([0, 5])
    return qml.counts(wires=[0, 1, 2, 3, 4, 5])

In [3]:

# get simulation and real result for 6-GHZ
sim_qnode = qml.QNode(ghz6, qml.device("default.qubit", shots=1000))
sim_results = sim_qnode()
sim_samples = []
for key in sim_results:
    count = sim_results[key]
    for _ in range(count):
        sim_samples.append(key)

path = os.path.join(os.getcwd(), ".env")

client = MonarqClient("https://manager.anyonlabs.com", 
                      "stage",
                      "FjjIKjmDMoAMzSO4v2Bu62a+8vD39zib")
monarq_dev = MonarqDevice(shots=1000, client=client, behaviour_config=TranspilerConfig())
transpiler = Transpiler.get_transpiler(TranspilerConfig())
real_tape = transpiler(sim_qnode.tape)[0][0]

real_qnode = qml.QNode(lambda: arbitrary_circuit(real_tape, measurement=qml.counts), monarq_dev)
real_results = real_qnode()
real_samples = []
for key in real_results:
    count = real_results[key]
    for _ in range(count):
        real_samples.append(key)

TypeError: __str__ returned non-string (type NoneType)

In [4]:
# get wires 
wires = [w for w in real_qnode.tape.wires]

print(wires)

[4, 12, 8, 5, 1, 0]


In [5]:
# get noise model for each wire
noise_model : dict[int, float] = {w:0 for w in wires}
for i, w in enumerate(wires):
    for real in real_samples:
        one_count = sum([1 for r in real if r == '1'])
        if one_count > len(wires)/2 and real[i] == '1':
            noise_model[w] += 1
        elif one_count < len(wires)/2 and real[i] == '0':
            noise_model[w] += 1
    noise_model[w] /= 1000

print(noise_model)

{4: 0.773, 12: 0.687, 8: 0.665, 5: 0.668, 1: 0.63, 0: 0.661}


In [14]:
# make another 6 wires circuit
grover = lambda : circuits.bernstein_vazirani(22)
grover_node = qml.QNode(grover, qml.device("default.qubit", shots=1000))
grover_node()
grover_tape = grover_node.tape
grover_tape = transpiler(grover_tape)[0][0]


In [17]:
# get results for circuit
grover_node = qml.QNode(lambda: arbitrary_circuit(grover_tape, measurement=qml.counts), monarq_dev)
grover_results = grover_node()
grover_samples = []
for key in grover_results:
    count = grover_results[key]
    for _ in count[()] if isinstance(count, np.ndarray) else count:
        grover_samples.append(key)

TypeError: iteration over a 0-d array

In [ ]:
# bitflip result according to noise model
